# 理解MNIST例子

## 序言

载入必要的库文件

In [1]:
import time
import numpy as np
import tensorflow as tf
import tensorlayer as tl

In [2]:
tf.logging.set_verbosity(tf.logging.DEBUG)
tl.logging.set_verbosity(tl.logging.DEBUG)

sess = tf.InteractiveSession()

## 加载训练数据集

In [3]:
# prepare data
X_train, y_train, X_val, y_val, X_test, y_test = tl.files.load_mnist_dataset(shape=(-1, 784))

print("X_train.shape:{}".format(X_train.shape))
print("y_train.shape:{}".format(y_train.shape))
print("X_val.shape:{}".format(X_val.shape))
print("y_val.shape:{}".format(y_val.shape))
print("X_test.shape:{}".format(X_test.shape))
print("y_test.shape:{}".format(y_test.shape))

[TL] Load or Download MNIST > data\mnist
[TL] data\mnist\train-images-idx3-ubyte.gz
[TL] data\mnist\t10k-images-idx3-ubyte.gz
X_train.shape:(50000, 784)
y_train.shape:(50000,)
X_val.shape:(10000, 784)
y_val.shape:(10000,)
X_test.shape:(10000, 784)
y_test.shape:(10000,)


`X_train.shape` 为 `(50000,784)`，可以理解成共有50000张图片并且每张图片有784个数值（像素点）。 `Y_train.shape` 为 `(50000,)` ，它是一个和 `X_train` 长度相同的向量，用于给出每幅图的数字标签，即这些图片所包含的位于0-9之间的10个数字。

另外对于卷积神经网络的例子，MNIST还可以按下面的4D版本来载入：

X_train, y_train, X_val, y_val, X_test, y_test = tl.files.load_mnist_dataset(shape=(-1, 28, 28, 1))

`X_train.shape` 为 `(50000,28,28,1)` ，这代表了50000张图片，每张图片有28行和28列。 通道为1是因为它是灰度图像，所以每个像素只能有一个值。

## 定义placeholder

In [4]:
# define placeholder
x = tf.placeholder(tf.float32, shape=[None, 784], name='x')
y_ = tf.placeholder(tf.int64, shape=[None], name='y_')

## 定义模型

In [5]:
# define the network
network = tl.layers.InputLayer(x, name='input')
network = tl.layers.DropoutLayer(network, keep=0.8, name='drop1')
network = tl.layers.DenseLayer(network, 800, tf.nn.relu, name='relu1')
network = tl.layers.DropoutLayer(network, keep=0.5, name='drop2')
network = tl.layers.DenseLayer(network, 800, tf.nn.relu, name='relu2')
network = tl.layers.DropoutLayer(network, keep=0.5, name='drop3')
# the softmax is implemented internally in tl.cost.cross_entropy(y, y_) to
# speed up computation, so we use identity here.
# see tf.nn.sparse_softmax_cross_entropy_with_logits()
network = tl.layers.DenseLayer(network, n_units=10, act=None, name='output')

[TL] InputLayer  input: (?, 784)
[TL] DropoutLayer drop1: keep: 0.800000 is_fix: False
[TL] DenseLayer  relu1: 800 relu
[TL] DropoutLayer drop2: keep: 0.500000 is_fix: False
[TL] DenseLayer  relu2: 800 relu
[TL] DropoutLayer drop3: keep: 0.500000 is_fix: False
[TL] DenseLayer  output: 10 No Activation


## 定义损失函数和衡量指标

In [6]:
# define cost function and metric.
y = network.outputs
cost = tl.cost.cross_entropy(y, y_, name='cost')
correct_prediction = tf.equal(tf.argmax(y, 1), y_)
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
y_op = tf.argmax(tf.nn.softmax(y), 1)

In [7]:
# define the optimizer
train_params = network.all_params
train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost, var_list=train_params)

In [8]:
# initialize all variables in the session
tl.layers.initialize_global_variables(sess)

[TL] WARNING: From C:\Users\Zach\Anaconda3\envs\tensorflow\lib\site-packages\wrapt\wrappers.py:523: initialize_global_variables (from tensorlayer.layers.utils) is deprecated and will be removed after 2018-09-30.
Instructions for updating: This API is deprecated in favor of `tf.global_variables_initializer`



In [9]:
# print network information
network.print_params()
network.print_layers()

[TL]   param   0: relu1/W:0            (784, 800)         float32_ref (mean: 0.00010284743621014059, median: 3.435601684032008e-05, std: 0.08803313970565796)   
[TL]   param   1: relu1/b:0            (800,)             float32_ref (mean: 0.0               , median: 0.0               , std: 0.0               )   
[TL]   param   2: relu2/W:0            (800, 800)         float32_ref (mean: 0.0001710604119580239, median: 0.00013219917309470475, std: 0.08801638334989548)   
[TL]   param   3: relu2/b:0            (800,)             float32_ref (mean: 0.0               , median: 0.0               , std: 0.0               )   
[TL]   param   4: output/W:0           (800, 10)          float32_ref (mean: -0.0009490904631093144, median: -0.00134329404681921, std: 0.08757556974887848)   
[TL]   param   5: output/b:0           (10,)              float32_ref (mean: 0.0               , median: 0.0               , std: 0.0               )   
[TL]   num of params: 1276810
[TL]   layer   0: x:0        

In [10]:
# train the network
tl.utils.fit(sess, network, train_op, cost, X_train, y_train, x, y_, acc=acc, batch_size=500, \
    n_epoch=500, print_freq=5, X_val=X_val, y_val=y_val, eval_train=False)

[TL] Start training the network ...
[TL] Epoch 1 of 500 took 0.825790s
[TL]    val loss: 0.567263
[TL]    val acc: 0.818300
[TL] Epoch 5 of 500 took 0.557472s
[TL]    val loss: 0.289096
[TL]    val acc: 0.912000
[TL] Epoch 10 of 500 took 0.560538s
[TL]    val loss: 0.227771
[TL]    val acc: 0.934200
[TL] Epoch 15 of 500 took 0.559512s
[TL]    val loss: 0.192987
[TL]    val acc: 0.947100
[TL] Epoch 20 of 500 took 0.559504s
[TL]    val loss: 0.168367
[TL]    val acc: 0.954500
[TL] Epoch 25 of 500 took 0.560499s
[TL]    val loss: 0.148946
[TL]    val acc: 0.960200
[TL] Epoch 30 of 500 took 0.558468s
[TL]    val loss: 0.134236
[TL]    val acc: 0.964400
[TL] Epoch 35 of 500 took 0.558469s
[TL]    val loss: 0.123387
[TL]    val acc: 0.966000
[TL] Epoch 40 of 500 took 0.554483s
[TL]    val loss: 0.114395
[TL]    val acc: 0.968800
[TL] Epoch 45 of 500 took 0.556512s
[TL]    val loss: 0.106742
[TL]    val acc: 0.969900
[TL] Epoch 50 of 500 took 0.557485s
[TL]    val loss: 0.099622
[TL]    val a

[TL]    val acc: 0.986100
[TL] Epoch 455 of 500 took 0.554517s
[TL]    val loss: 0.056411
[TL]    val acc: 0.986100
[TL] Epoch 460 of 500 took 0.553558s
[TL]    val loss: 0.056310
[TL]    val acc: 0.986000
[TL] Epoch 465 of 500 took 0.555547s
[TL]    val loss: 0.055954
[TL]    val acc: 0.986000
[TL] Epoch 470 of 500 took 0.556512s
[TL]    val loss: 0.056451
[TL]    val acc: 0.986200
[TL] Epoch 475 of 500 took 0.555510s
[TL]    val loss: 0.055543
[TL]    val acc: 0.986800
[TL] Epoch 480 of 500 took 0.558540s
[TL]    val loss: 0.056381
[TL]    val acc: 0.986700
[TL] Epoch 485 of 500 took 0.551525s
[TL]    val loss: 0.055536
[TL]    val acc: 0.986200
[TL] Epoch 490 of 500 took 0.555514s
[TL]    val loss: 0.055955
[TL]    val acc: 0.986500
[TL] Epoch 495 of 500 took 0.555560s
[TL]    val loss: 0.056649
[TL]    val acc: 0.986900
[TL] Epoch 500 of 500 took 0.556512s
[TL]    val loss: 0.056485
[TL]    val acc: 0.986500
[TL] Total training time: 287.444314s


In [11]:
# evaluation
tl.utils.test(sess, network, acc, X_test, y_test, x, y_, batch_size=None, cost=cost)

[TL] Start testing the network ...
[TL]    test loss: 0.051042
[TL]    test acc: 0.987400


In [12]:
# save the network to .npz file
tl.files.save_npz(network.all_params, name='model.npz')
sess.close()

[TL] [*] model.npz saved
